In [ ]:
!pip install -r ../requirements.txt

     |████████████████████████████████| 45 kB 2.0 MB/s 
     |████████████████████████████████| 311 kB 8.7 MB/s 
     |████████████████████████████████| 3.5 MB 79.2 MB/s 
     |████████████████████████████████| 1.1 MB 67.5 MB/s 
     |████████████████████████████████| 243 kB 90.0 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 133 kB 93.0 MB/s 
     |████████████████████████████████| 312 kB 100.3 MB/s 
     |████████████████████████████████| 311 kB 76.3 MB/s 
     |████████████████████████████████| 311 kB 81.9 MB/s 
     |████████████████████████████████| 306 kB 76.3 MB/s 
     |████████████████████████████████| 298 kB 83.7 MB/s 
     |████████████████████████████████| 298 kB 76.5 MB/s 
     |████████████████████████████████| 290 kB 79.2 MB/s 
     |████████████████████████████████| 290 kB 77.2 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 290 kB 94.7 MB/s 
     |██████████████

In [ ]:
model_checkpoint = "google/pegasus-xsum"

In [ ]:
from datasets import load_metric

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
import pandas as pd
df = pd.read_csv("../data/filtered/train_pairs.csv", index_col=0)
df

,title,abstract,title_length,abstract_length
0,Natural Image Bases to Represent Neuroimaging ...,Visual inspection of neuroimagery is susceptib...,7,132
1,Sluice Resolution without Hand-Crafted Feature...,Sluice resolution in English is the problem of...,9,110
2,Learning Translation Models from Monolingual C...,Translation models often fail to generate good...,7,152
3,Sentiment Adaptive End-to-End Dialog Systems,End-to-end learning framework is useful for bu...,5,119
4,User-Friendly Text Prediction For Translators,Text prediction is a form of interactive machi...,5,134
...,...,...,...,...
21437,"Arabic Tokenization, Part-of-Speech Tagging an...",We present an approach to using a morphologica...,11,58
21438,Using Semantically Motivated Estimates to Help...,Research into the automatic acquisition of sub...,8,530
21439,A Mathematical Exploration of Why Language Mod...,"Autoregressive language models, pretrained usi...",11,194
21440,Do You Know That Florence Is Packed with Visit...,"When a speaker, Mary, asks ""Do you know that F...",15,174


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv("../data/filtered/1024_characters_pairs.csv")

In [ ]:
df = df.drop(columns=["title_length", "abstract_length", "token_len"])

In [ ]:
df_train = df[:11864]
df_valid = df[11864:]

In [ ]:
from datasets import Dataset
from datasets import load_dataset, load_metric
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
metric = load_metric("rouge")


In [ ]:
df_train

,title,abstract
0,Natural Image Bases to Represent Neuroimaging ...,Visual inspection of neuroimagery is susceptib...
1,Sluice Resolution without Hand-Crafted Feature...,Sluice resolution in English is the problem of...
2,Sentiment Adaptive End-to-End Dialog Systems,End-to-end learning framework is useful for bu...
3,User-Friendly Text Prediction For Translators,Text prediction is a form of interactive machi...
4,Aligning Sentences from Standard Wikipedia to ...,This work improves monolingual sentence alignm...
...,...,...
11859,Vine Parsing and Minimum Risk Reranking for Sp...,We describe our entry in the CoNLL-X shared ta...
11860,From Characters to Time Intervals: New Paradig...,This paper presents the first model for time n...
11861,Unsupervised Consonant-Vowel Prediction over H...,"In this paper, we present a solution to one as..."
11862,Variational Autoencoder with Arbitrary Conditi...,We propose a single neural probabilistic model...


In [ ]:
max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    model_inputs = tokenizer(examples["abstract"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
preprocess_function(valid_dataset[:2])


{'input_ids': [[222, 1172, 1083, 110, 81576, 108, 114, 327, 5086, 112, 1735, 109, 1083, 113, 114, 1172, 135, 23455, 556, 107, 7087, 7332, 112, 563, 114, 281, 121, 23452, 1172, 1083, 110, 81576, 327, 444, 109, 5200, 113, 20076, 335, 118, 136, 1778, 111, 113, 18972, 1226, 121, 40042, 1083, 35760, 107, 507, 618, 6582, 9417, 135, 109, 617, 120, 109, 1778, 117, 270, 2839, 115, 10530, 135, 433, 1481, 113, 2093, 8240, 32799, 16200, 8389, 335, 107, 222, 136, 800, 108, 145, 1037, 109, 985, 1778, 113, 1172, 5256, 108, 241, 145, 1216, 112, 1735, 109, 1794, 5256, 113, 114, 1172, 135, 2956, 107, 184, 137, 207, 6529, 1261, 110, 88758, 130, 114, 423, 1376, 113, 8621, 11559, 335, 118, 136, 1778, 107, 184, 799, 8970, 118, 6676, 109, 1172, 5256, 575, 111, 4454, 114, 1225, 2757, 204, 114, 13757, 327, 107, 184, 237, 403, 120, 109, 1172, 121, 57064, 327, 137, 129, 263, 112, 927, 637, 124, 114, 15388, 1157, 57064, 1778, 111, 137, 2448, 111, 7127, 31044, 109, 323, 113, 3194, 16207, 118, 114, 1172, 107, 1], [

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
batch_size = 3
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-lm_al_paper",
    evaluation_strategy = "epoch",
    learning_rate=6e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=8,
    save_steps = 1000,
    logging_steps = 185,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)





```
Examples:
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> bertscore = datasets.load_metric("bertscore")
    >>> results = bertscore.compute(predictions=predictions, references=references, lang="en")
    >>> print([round(v, 2) for v in results["f1"]])
    [1.0, 1.0]
  
```



In [ ]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
 trainer.train()

The following columns in the training set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: title, abstract.
***** Running training *****
  Num examples = 11864
  Num Epochs = 3
  Instantaneous batch size per device = 3
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 8
  Total optimization steps = 1482


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,2.773300,2.313878,45.731300,25.526500,40.648700,40.680800,11.575000
1,2.062500,2.190471,47.518300,27.083900,42.429700,42.477200,11.111000
2,1.552600,2.233600,47.666300,26.851400,42.389900,42.413100,11.816000


The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: title, abstract.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 2
The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: title, abstract.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 2
Saving model checkpoint to pegasus-xsum-finetuned-lm_al_paper/checkpoint-1000
Configuration saved in pegasus-xsum-finetuned-lm_al_paper/checkpoint-1000/config.json
Model weights saved in pegasus-xsum-finetuned-lm_al_paper/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in pegasus-xsum-finetuned-lm_al_paper/checkpoint-1000/tokenizer_config.json
Special tokens file saved in pegasus-xsum-finetuned-lm_al_paper/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set  don't have a corres

TrainOutput(global_step=1482, training_loss=2.172951754931657, metrics={'train_runtime': 4187.0167, 'train_samples_per_second': 8.501, 'train_steps_per_second': 0.354, 'total_flos': 1.8147817384525824e+16, 'train_loss': 2.172951754931657, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./output/bigbird-pegasus")


Configuration saved in ./output/bigbird-pegasus/config.json
Model weights saved in ./output/bigbird-pegasus/pytorch_model.bin


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
model = AutoModelForSeq2SeqLM.from_pretrained("../output/bigbird-pegasus")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model.to("cuda")

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementw

In [ ]:
abstract-to-title-generation

In [ ]:
import pandas as pd

test_samples = pd.read_csv("../data/filtered/test_pairs.csv", index_col=0)
test_samples

,title,abstract,title_length,abstract_length
0,Learning Latent Semantic Annotations for Groun...,Previous work on grounded language learning di...,11,121
1,Partially Supervised Sense Disambiguation by L...,Supervised and semi-supervised sense disambigu...,13,140
2,Hawkes Processes for Continuous Time Sequence ...,Classification of temporal textual data sequen...,15,68
3,A Unified Single Scan Algorithm for Japanese B...,We describe an algorithm for Japanese analysis...,13,62
4,Generating Coherent Event Schemas at Scale,Chambers and Jurafsky (2009) demonstrated that...,6,127
...,...,...,...,...
5356,Bridging Information-Seeking Human Gaze and Ma...,"In this work, we analyze how human gaze during...",8,118
5357,Quantum-inspired Neural Network for Conversati...,We provide a novel perspective on conversation...,7,116
5358,The BQ Corpus: A Large-scale Domain-specific C...,This paper introduces the Bank Question (BQ) c...,13,174
5359,Doc2hash: Learning Discrete Latent variables f...,Learning to hash via generative model has beco...,8,131


In [ ]:
def creat_eval_pairs(model, tokenizer, abstracts, titles):
  preds = []
  for abstract, title in zip(abstracts, titles):
    encoding = tokenizer.encode_plus(abstract, return_tensors = "pt")
    inputs = encoding["input_ids"].to("cuda")
    attention_masks = encoding["attention_mask"].to("cuda")
    title_ids = model.generate(
            input_ids = inputs,
            attention_mask = attention_masks,
            max_length = 30,
            num_beams = 5,
            num_return_sequences = 5,
            repetition_penalty=2.0, 
            length_penalty=10.0,
            early_stopping = True,
            )
    result = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in title_ids]
    s=""
    for t in result:
      s = s + "<TITLE>" + t
    preds.append(s)
    if len(preds) % 500 == 0:
      print("original title: ", title)
      print("generated title: ", preds[-1:])
  return preds, titles

In [ ]:
abstracts = test_samples.abstract.to_list()
titles = test_samples.title.to_list()

In [ ]:
preds, titles = creat_eval_pairs(model, tokenizer, abstracts, titles)

original title:  Paraphrase-Driven Learning for Open Question Answering
generated title:  ['<TITLE>Learning a Semantic Lexicon and Linear Ranking Function for Question Answering<TITLE>Learning a Semantic Lexicon and Ranking Function for Question Answering<TITLE>Learning a Lexicon and Ranking Function for Question Answering<TITLE>Question Answering as a Machine Learning Problem<TITLE>Learning to Answer Open-Domain Questions']
original title:  Robustness and Generalization of Role Sets: PropBank vs. VerbNet
generated title:  ['<TITLE>An Empirical Study on the Robustness of VerbNet and PropBank Roles for Semantic Role Labeling<TITLE>An Empirical Study on the Robustness of PropositionBank and VerbNet Roles for Semantic Role Labeling<TITLE>An Empirical Study on the Robustness and Semantic Role Labeling Performance of PropositionBank and VerbNet<TITLE>An Empirical Study on the Robustness and Semantic Role Labeling of PropositionBank and VerbNet Roles<TITLE>An Empirical Study on the Robustnes

In [ ]:
pred_target_pairs = pd.DataFrame(list(zip(preds, titles)), columns=['predictions', 'targets'])

In [ ]:
pred_target_pairs.to_csv("../output/preds_targets_pairs/pegasus.csv")